# Cooling with recycling

Contents
- 1. **General model**

    - 1.1. Building characteristics and auxiliary loads
    
    - 1.2. Create air handing unit (AHU) object
    
- 2. **System without reaheating ($K_w = 0$)**

    - 2.1. CAV Constant Air Volume
    
    - 2.2. VAV Variable Air Volume
    
        2.2.1. Control of supply air temperature by supply air mass flow rate
        
        2.2.2. Control of indoor air humidity by supply air mass flow rate

    - 2.3. Variable by-pass of the cooling coil
    
        2.3.1. CAV with control of supply air temperature $\theta_S$
        
        2.3.2. CAV control of indoor air humidity
        
-  3. **System with reheating**

    - 3.1. CAV: control indoor temperature & humidity

    - 3.2. VAV: control indoor temperature & humidity and supply temperature

--------------------

Model: 
- Mixing (recirculating), cooling coil with by-pass, reheating, thermal zone, building. 
- CAV (constant air volume): the mass flow rate of supply air is constant.
- VAV (variable air volume): the mass flow rate of supply air is variable.
- VBP (variable by-pass): variable by-pass of cooling coil with constant supply air the mass flow rate. 

Control:
1. Indoor temperature controlled by the cooling-coil total heat flow rate (the supply air mass flow rate is given). Indoor humidity is not controlled.
2. Indoor temperature controlled by cooling coil; supply air temperature controlled by mass flow rate.
3. Indoor temperature controlled by cooling coil; indoor air humidity controlled by the mass flow rate.
4. Indoor temperature controlled by cooling coil; idoor air umidity controlled by the cooling-coil by-pass.

In [1]:
import ipywidgets as wd
import matplotlib.pyplot as plt
# %matplotlib inline  # uncomment for inline figure
# %matplotlib qt      # uncomment for figure in separate window
# plt.show()

plt.rcParams["figure.figsize"] = (10, 7.7)
font = {'size': 16}
plt.rc('font', **font)

In [2]:
import cool as cc

## General model

### Building characteristics and auxiliary loads

Total:
- sensible: 45 kW
- latent: 15 kW

Rules of thumb, BSRIA 2003
Solar gains over floor area 150-250 W/m2 of glass (Tab. 3/p 13)
Heat loads 50 - 100 W/m2 floor area
U-wall 0.4 W/m2K
U-window 3.5 W/m2K

In [3]:
import psychro as psy
# constants
c = 1e3         # J/kg K, air specific heat
l = 2496e3      # J/kg, latent heat
ρ = 1.2         # kg/m3, density

# Building dimensions
length = 20     # m
width = 30      # m
height = 3.5    # m
persons = 100   # m

sens_heat_person = 60       # W / person
latent_heat_person = 40     # W / person
load_m2 = 15        # W/m2
solar_m2 = 150      # W/m2 of window area
ACH = 1             # Air Cnhnages per Hour
U_wall = 0.4        # W/K, overall heat transfer coeff. walls
U_window = 3.5      # W/K, overall heat transfer coeff. windows

θo, φo = 32, 0.5    # outdoor temperature & relative humidity
θI, φI = 26, 0.5    # indoor temperature & relative humidity
wo = psy.w(θo, φo)
wI = psy.w(θI, φI)

floor_area = length * width
surface_floor = 2 * (length + width) * height + floor_area
surface_wall = 0.9 * surface_floor
surface_window = surface_floor - surface_wall

UA = U_wall * surface_wall + U_window * surface_window
mi = ACH * surface_floor * height / 3600 * ρ

solar_gains = solar_m2 * surface_window
electrical_load = load_m2 * surface_floor
Qsa = persons * sens_heat_person + solar_gains + electrical_load
Qla = persons * latent_heat_person

QsTZ = (UA + mi * c) * (θo - θI) + Qsa
QlTZ = mi * l * (wo - wI) + Qla

θS = θI - 15        # °C supply air temperature
m = QsTZ / c / (θI - θS)

print(f'QsTZ = {QsTZ:.0f} W, QlTZ = {QlTZ:.0f} W')
print(f'UA = {UA:.0f} W/K, mi = {mi:.2f} kg/s,\
      Qsa = {Qsa:.0f} W, Qla = {Qla:.0f} W')
print(f'm = {m:.3f} kg/s')

QsTZ = 45197 W, QlTZ = 16338 W
UA = 674 W/K, mi = 1.11 kg/s,      Qsa = 34500 W, Qla = 4000 W
m = 3.013 kg/s


### Create air handing unit (AHU) object

In [4]:
Kθ, Kw = 1e10, 0     # Kw can be 0
β = 0.2              # by-pass factor

m, mo = 3.1, 1.      # kg/s, mass flow rate: supply & outdoor (fresh) air
θo, φo = 32., 0.5    # outdoor conditions
θIsp, φIsp = 26., 0.5        # set point for indoor condition

mi = 1.35            # kg/s, mass flow rate of infiltration air
UA = 675.            # W/K, overall heat coefficient of the building
Qsa, Qla = 34000., 4000.     # W, auxiliary loads: sensible & latent

parameters = m, mo, β, Kθ, Kw
inputs = θo, φo, θIsp, φIsp, mi, UA, Qsa, Qla

# System without reaheating ($K_w = 0$)
## CAV Constant Air Volume

<img src="Figures/cool_CAV.png" alt="cool_CAV.png" style="width: 600px;"/>

> Constant air volume (CAV) cooling system without reheating (the sensible heat flow rate of the heating coil HC is zero, $\dot{Q}_{sHC}=0$; the indoor humidity $\varphi_4$ is not controlled, $K_w = 0$). Controlled variables: supply air temperature $\theta_3 = \theta_4$ and indoor air temperature $\theta_4$. 

In [11]:
cool1 = cc.MxCcRhTzBl(parameters, inputs)
wd.interact(cool1.CAV_wd, θo=(26, 34), φo=(0.4, 1), θIsp=(20, 28), φIsp=(0.4, 1),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), QsBL=(0, 60_000, 500), QlBL=(0, 20_000, 500));

interactive(children=(IntSlider(value=32, description='θo', max=34, min=26), FloatSlider(value=0.8, descriptio…

> CAV cooling without reheating with control of indoor temperature. Points 3 and 4 are superposed on the pshychrometric diagram.

# VAV Variable Air Volume

## Control of supply air temperature by supply air mass flow rate

<img src="Figures/cool_VAV_θS.png" alt="cool_VAV_θS.png" style="width: 600px;"/>

> Variable air volume (VAV) cooling system without reheating (the sensible heat flow rate of the heating coil HC is zero, $\dot{Q}_{sHC}=0$).  The indoor humidity $\varphi_4$ is not controlled ($K_w = 0$). Controlled variables: supply air temperature $\theta_3$ and indoor air temperature $\theta_4$.

In [12]:
cool2 = cc.MxCcRhTzBl(parameters, inputs)
wd.interact(cool2.VAV_wd, value='θS', sp=(14, 21), θo=(28, 36), φo=(0.4, 1),
            θIsp=(22, 26), φIsp=(0.4, 0.8),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), QsBL=(0, 60_000, 500), QlBL=(0, 20_000, 500));

interactive(children=(Text(value='θS', description='value'), IntSlider(value=18, description='sp', max=21, min…

> VAV cooling without reheating with control of indoor temperature and supply air. Points 3 and 4 are superposed on the pshychrometric diagram.

### Control of indoor air humidity by supply air mass flow rate

<img src="Figures/cool_VAV_φI.png" alt="cool_VAV_φI.png" style="width: 600px;"/>

> Variable air volume (VAV) cooling system without reheating (the sensible heat flow rate of the heating coil HC is zero, $\dot{Q}_{sHC}=0$).  The indoor humidity $\varphi_5$ is controlled by a non-linear (least-sqaures) controller. Controlled variable: indoor air temperature $\theta_4$.

The indoor air humidity is controlled by the mass flow rate of dry air, $\dot{m}_{da}$ in the Air Handing Unit (AHU).
The heating coil HC is not used (*KwI* = 0).

In [13]:
cool3 = cc.MxCcRhTzBl(parameters, inputs)
wd.interact(cool3.VAV_wd, value='φI', sp=(0.4, 0.8), θo=(28, 36), φo=(0.4, 1),
            θIsp=(22, 26), φIsp=(0.4, 0.8),
            mi=(0.5, 3, 0.05), UA=(500, 800, 10), Qsa=(0, 60_000, 500), Qla=(0, 20_000, 500));

interactive(children=(Text(value='φI', description='value'), FloatSlider(value=0.8, description='sp', max=0.8,…

> VAV cooling without reheating with control of indoor temperature and humidity. Points 3 and 4 are superposed on the pshychrometric diagram.

# Variable by-pass of the cooling coil

## CAV with control of supply air temperature $\theta_S$

<img src="Figures/cool_CAVb_θS.png" alt="cool_CAVb_θS.png" style="width: 600px;"/>

> Constant air volume (CAV) cooling system without reheating (the sensible heat flow rate of the heating coil HC is zero, $\dot{Q}_{sHC}=0$).  The supply air temperature $\theta_4$ is controlled by a non-linear (least-sqaures) controller. Controlled variable: indoor air temperature $\theta_4$.

> **Note**: The supply air temperature, $\theta_S$, cannot be controlled when $\dot{m}$, $\theta_I$ and $\dot{Q}_{sTZ}$ are imposed. This results from the sensible heat balance on the thermal zone: $\dot{m}c\theta_S - \dot{m}c\theta_I + \dot{Q}_{sTZ} = 0$

## CAV control of indoor air humidity
                         
<img src="Figures/cool_CAVb_φI.png" alt="cool_CAVb_φI.png" style="width: 600px;"/>

> Constant air volume (CAV) cooling system without reheating (the sensible heat flow rate of the heating coil HC is zero, $\dot{Q}_{sHC}=0$). The indoor air humidity $\varphi_5$ is controlled by a non-linear (least-sqaures) controller. Controlled variable: indoor air temperature $\theta_4$.


In [14]:
cool4 = cc.MxCcRhTzBl(parameters, inputs)
wd.interact(cool4.VBP_wd, value='φI', sp=(0.2, 0.6), θo=(28, 36), φo=(0.4, 1),
            θIsp=(22, 26), φIsp=(0.4, 0.8),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), Qsa=(0, 60_000, 500), Qla=(0, 20_000, 500));

interactive(children=(Text(value='φI', description='value'), FloatSlider(value=0.6, description='sp', max=0.6,…

**Note**: By changing the initial guess for $\beta$ in `β_ls(self, value, sp)`, a second solution may be found (i.e. the second secant point of the saturation curve).  

## System with reheating

### CAV: control indoor temperature & humidity

<img src="Figures/cool_CAV.png" alt="cool_CAV.png" style="width: 600px;"/>

> Constant air volume (CAV) cooling system with reheating. The indoor air temperature $\theta_4$ and humidity $\varphi_4$ are controlled.

In [15]:
cool5 = cc.MxCcRhTzBl(parameters, inputs)
β, Kw = 0.2, 1e10
cool5.actual[[2, 4]] = β, Kw
wd.interact(cool5.CAV_wd, θo=(26, 34), φo=(0.4, 1), θIsp=(20, 28), φIsp=(0.4, 1),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), Qsa=(0, 60_000, 500), Qla=(0, 20_000, 500));

interactive(children=(IntSlider(value=32, description='θo', max=34, min=26), FloatSlider(value=0.8, descriptio…

## VAV: control indoor temperature & humidity and supply temperature
 
<img src="Figures/cool_VAV_θS.png" alt="cool_VAV_θS.png" style="width: 600px;"/>

> Variable air volume (VAV) cooling system with reheating. The indoor air temperature $\theta_4$ and humidity $\varphi_4$ and the supply air temperature $\theta_4$ are controlled.

In [16]:
cool6 = cc.MxCcRhTzBl(parameters, inputs)
Kw = 1e10
cool6.actual[4] = Kw
wd.interact(cool6.VAV_wd, value='θS', sp=(14, 21), θo=(28, 36), φo=(0.4, 1),
            θIsp=(22, 26), φIsp=(0.4, 0.8),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), Qsa=(0, 60_000, 500), Qla=(0, 20_000, 500));

interactive(children=(Text(value='θS', description='value'), IntSlider(value=18, description='sp', max=21, min…